In [1]:
!pip install PyPDF2
!pip install langchain
!pip install torch
!pip install acclerate
!pip install sentence_transformers
!pip install streamlit_chat
!pip install streamlit
!pip install faiss-cpu
!pip install tiktoken
!pip install huggingface-hub
!pip install pypdf
!pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.3 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement acclerate (from versions: none)
ERROR: No matching distribution found for acclerate
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.3 MB/s eta 0:00:00
     ━━━━━━━

In [2]:
%%writefile app.py
import streamlit as st
from streamlit_chat import message
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import LlamaCpp
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.document_loaders import PyPDFLoader
from PyPDF2 import PdfReader
import os
import tempfile

def get_pdf_text(pdf_docs):
    text = ''
    for pdf in pdf_docs:
        pdf_reader = PdfReader(pdf)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text
def get_text_chunks(text):
    text_splitter = CharacterTextSplitter(separator='\n', chunk_size=1000, chunk_overlap=200,length_function=len)
    chunks = text_splitter.split_text(text)
    return chunks
def get_vectorstore(text_chunks):
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',model_kwargs={'device':'cpu'})
    vectorstore = FAISS.from_texts(texts=text_chunks,embedding=embeddings)
    return vectorstore
def get_conversation_chain(vectorstore):
    llm = LlamaCpp(streaming=True,
               model_path='/content/drive/MyDrive/mistral-7b-instruct-v0.1.Q4_K_M.gguf',
               temperature=0.75,
               top_p=1,
               verbose=True,
               n_ctx=4096)
    memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
    conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm,retriever=vectorstore.as_retriever(),memory=memory)
    return conversation_chain
def handle_userinput(user_question):
    response=st.session_state.conversation({'question': user_question})
    st.write(response)





def main():

    st.set_page_config(page_title="CHAT WITH YOUR PDF DOCUMENT",page_icon=":books:")
    if 'conversation' not in st.session_state:
        st.session_state.conversation = None
    st.header("CHAT WITH YOUR PDF DOCUMENT :books:")
    user_question = st.text_input("ASK QUESTION:")
    if user_question:
        handle_userinput(user_question)
    with st.sidebar:
        st.subheader('DOCUMENTS')
        pdf_docs=st.file_uploader('UPLOAD THE PDF & CLICK ON PROCESS',accept_multiple_files=True)
        if st.button('PROCESS'):
            with st.spinner('PROCESSING'):

               # get pdf text
               raw_text = get_pdf_text(pdf_docs)
               # get the text chunks
               text_chunks = get_text_chunks(raw_text)
               # create vector store
               vectorstore = get_vectorstore(text_chunks)
               # create conversation chain
               st.session_state.conversation = get_conversation_chain(vectorstore)

if __name__ == '__main__':
    main()

Writing app.py


In [ ]:
! streamlit run app.py & npx localtunnel --port 8501

[#.................] - fetchMetadata: sill resolveWithNewModule follow-redirect


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.16.181.91:8501

npx: installed 22 in 4.787s
your url is: https://six-ducks-talk.loca.lt
llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /content/drive/MyDrive/mistral-7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q4_K     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight q4_K     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight q6_K     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight q4_K     [  4096,  4096,     1,     1 ]
